In [4]:
!pip install tmdbsimple
!pip install python-dateutil
import numpy as np
import pandas as pd
import tmdbsimple as tmdb
import time
import datetime
from dateutil import parser

In [5]:
tmdb.API_KEY = '0b6ab414e6fd79648a868782b0750b09'

In [6]:
tv = tmdb.TV(456)
info = tv.info()

In [7]:
genres = info.get('genres')

In [8]:
for x in genres:
    print(x['name'])

Animation
Comedy


In [25]:
tv_id = 1
shows = []
ratings = []
genres = []
seasons = []
pops = []
episodes = []
ids = []

while (tv_id < 30000):
    tv = tmdb.TV(tv_id)
    time.sleep(.26)
    
    try:
        
        info = tv.info()
        name = info.get('name')
        season_no = info.get('number_of_seasons')
        episode = info.get('number_of_episodes')
        rating = info.get('vote_average')
        genre_s = info.get('genres')
        genre = ""
        pop = info.get('popularity')
        votes = info.get('vote_count')
        for x in genre_s:
            genre += x['name'] + "/"
        
        
        if votes >= 200 and season_no >= 4:
            shows.append(name)
            ids.append(tv_id)
            ratings.append(rating)
            genres.append(genre)
            episodes.append(episode)
            seasons.append(season_no)
            pops.append(pop)
            
    except:
        pass
    
    tv_id += 1
    

KeyboardInterrupt: 

In [26]:
print(shows)

['Top Gear', "That '70s Show", 'Buffy the Vampire Slayer', 'Weeds', 'Star Trek: Enterprise', 'The Simpsons', 'Futurama', 'Star Trek: The Next Generation', 'How I Met Your Mother', 'Mad Men', 'Californication', 'Six Feet Under', 'Breaking Bad', 'The Sopranos', 'Game of Thrones', 'Seinfeld', 'The Walking Dead', "Marvel's Agents of S.H.I.E.L.D.", 'Chuck', 'Dexter', 'Homeland', 'House', 'Sons of Anarchy', 'Person of Interest', 'Arrow', 'American Horror Story', 'Elementary', "Grey's Anatomy", 'The Big Bang Theory', 'Castle', 'New Girl', 'Modern Family', 'Ray Donovan', 'Orange Is the New Black', 'House of Cards', 'Luther', 'CSI: Crime Scene Investigation', 'American Dad!', 'Family Guy', 'The Wire', 'Boardwalk Empire', 'Supernatural', 'Heroes', 'Friends', 'Fringe', 'Star Trek: Voyager', 'The Fresh Prince of Bel-Air', 'Bones', 'Battlestar Galactica', '24', 'Malcolm in the Middle', 'South Park', 'Prison Break', 'Stargate Atlantis', 'The Office', 'The IT Crowd', 'Two and a Half Men', "It's Alway

In [27]:
print(len(shows))

72


In [28]:
print(tv_id)

17962


In [29]:
print(genres[0])

Documentary/Comedy/


In [30]:
print(len(genres))

72


In [31]:
with open("TVoutput.tsv", "a") as f:
    for x in range(len(shows)):
        print ("%s\t%d\t%d\t%d\t%d\t%d\t%s" % (shows[x], ids[x], ratings[x], seasons[x], episodes[x], pops[x], genres[x]), file=f)

In [32]:
df = pd.DataFrame.from_csv("TVoutput.tsv", sep='\t')

In [33]:
df

,45,7,24,176,27,Documentary/Comedy/
Top Gear,,,,,,
That '70s Show,52,7,8,200,19,Comedy/
Buffy the Vampire Slayer,95,8,7,144,13,Comedy/Drama/Sci-Fi & Fantasy/
Weeds,186,7,8,102,17,Drama/Comedy/
Star Trek: Enterprise,314,7,4,98,12,Action & Adventure/Drama/Sci-Fi & Fantasy/
The Simpsons,456,7,29,688,106,Animation/Comedy/
Futurama,615,8,7,124,20,Animation/Comedy/Drama/Sci-Fi & Fantasy/
Star Trek: The Next Generation,655,8,7,178,32,Action & Adventure/Drama/Mystery/Sci-Fi & Fant...
How I Met Your Mother,1100,7,9,208,30,Comedy/
Mad Men,1104,8,7,92,16,Drama/


In [35]:
with open("TVShow.tsv", "a") as f:
        print ("%s\t%s\t%s\t%s\t%s\t%s\t%s" % ("show", "tv_id", "rating", "seasons", "episodes", "popularity", "genres"), file=f)

In [36]:
with open("TVShow.tsv", "a") as f:
    for x in range(len(shows)):
        print ("%s\t%d\t%d\t%d\t%d\t%d\t%s" % (shows[x], ids[x], ratings[x], seasons[x], episodes[x], pops[x], genres[x]), file=f)

In [41]:
df1 = pd.DataFrame.from_csv("TVShow.tsv", sep='\t')

In [42]:
df1

,tv_id,rating,seasons,episodes,popularity,genres
show,,,,,,
Top Gear,45,7,24,176,27,Documentary/Comedy/
That '70s Show,52,7,8,200,19,Comedy/
Buffy the Vampire Slayer,95,8,7,144,13,Comedy/Drama/Sci-Fi & Fantasy/
Weeds,186,7,8,102,17,Drama/Comedy/
Star Trek: Enterprise,314,7,4,98,12,Action & Adventure/Drama/Sci-Fi & Fantasy/
The Simpsons,456,7,29,688,106,Animation/Comedy/
Futurama,615,8,7,124,20,Animation/Comedy/Drama/Sci-Fi & Fantasy/
Star Trek: The Next Generation,655,8,7,178,32,Action & Adventure/Drama/Mystery/Sci-Fi & Fant...
How I Met Your Mother,1100,7,9,208,30,Comedy/
